<a href="https://colab.research.google.com/github/DivyaLaxmi-div/Deep-Learning-Projects/blob/main/FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()   # Choose kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"divyalaxmirouthu","key":"0d74b9eeb4d4200ce2e904221eb33579"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset


Dataset URL: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset
License(s): CC-BY-NC-SA-4.0
fake-and-real-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip fake-and-real-news-dataset.zip


Archive:  fake-and-real-news-dataset.zip
replace Fake.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Fake.csv                
replace True.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd

fake = pd.read_csv("Fake.csv")
real = pd.read_csv("True.csv")

print(fake.head())
print(real.head())


                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  
                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept t

In [ ]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
# Step 2: Load Dataset
fake = pd.read_csv("Fake.csv")
real = pd.read_csv("True.csv")

In [ ]:
fake['label'] = 0   # Fake = 0
real['label'] = 1

In [ ]:
df = pd.concat([fake, real], axis=0).sample(frac=1).reset_index(drop=True)

In [ ]:
# Step 3: Split Data
x = df['text']
y = df['label']

In [ ]:
print(x)

0        PESHAWAR, Pakistan (Reuters) - A suicide bombe...
1        WASHINGTON (Reuters) - U.S. President-elect Do...
2        21st Century Wire says Here is a perfect examp...
3        THE HAGUE (Reuters) - A U.N. tribunal on Wedne...
4        You may have seen the latest Republican debate...
                               ...                        
44893    WASHINGTON (Reuters) - U.S. Republican preside...
44894                                                     
44895    At the very beginning of May, Trump s Departme...
44896    Kirsten Powers USA TodayOver the past 18 month...
44897    WASHINGTON (Reuters) - The United States on Tu...
Name: text, Length: 44898, dtype: object


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
# 1️⃣ Imports
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense

# 2️⃣ Parameters
voc_size = 5000
max_len = 20
embedding_dim = 100

# 3️⃣ One-hot encode the text
X_train_seq = [one_hot(text, voc_size) for text in X_train]
X_test_seq = [one_hot(text, voc_size) for text in X_test]

# 4️⃣ Pad sequences
X_train_pad = pad_sequences(X_train_seq, padding='pre', maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, padding='pre', maxlen=max_len)

In [ ]:
# 5️⃣ Build LSTM model
model = Sequential()
model.add(Embedding(voc_size, embedding_dim, input_length=max_len))
model.add(LSTM(128))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# 6️⃣ Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# 7️⃣ Fit model
history = model.fit(
    X_train_pad,
    y_train,
    validation_data=(X_test_pad, y_test),
    epochs=2,
    batch_size=64
)


Epoch 1/2
562/562 ━━━━━━━━━━━━━━━━━━━━ 50s 88ms/step - accuracy: 0.9708 - loss: 0.0858 - val_accuracy: 0.9637 - val_loss: 0.1056
Epoch 2/2
562/562 ━━━━━━━━━━━━━━━━━━━━ 76s 78ms/step - accuracy: 0.9844 - loss: 0.0464 - val_accuracy: 0.9638 - val_loss: 0.1147


In [ ]:
# 8️⃣ Check shapes (optional)
print("X_train_pad shape:", X_train_pad.shape)
print("X_test_pad shape:", X_test_pad.shape)

X_train_pad shape: (35918, 20)
X_test_pad shape: (8980, 20)


In [ ]:
# 9️⃣ Function to predict new text
def predict_fake_news(texts):
    """
    texts: list of strings (new news articles)
    returns: list of 0 (Not Fake) or 1 (Fake)
    """
    # One-hot encode new text
    seq = [one_hot(t, voc_size) for t in texts]
    # Pad sequences
    pad_seq = pad_sequences(seq, padding='pre', maxlen=max_len)
    # Predict probabilities
    pred_prob = model.predict(pad_seq)
    # Convert to classes 0/1
    pred_class = [1 if p > 0.5 else 0 for p in pred_prob]
    return pred_class, pred_prob

In [ ]:
# 10️⃣ Example prediction
new_articles = ["The stock market is crashing today!", "NASA discovered water on Mars."]
predicted_labels, predicted_prob = predict_fake_news(new_articles)

for i, text in enumerate(new_articles):
    print(f"Article: {text}")
    print(f"Prediction: {predicted_labels[i]} (Probability: {predicted_prob[i][0]:.4f})")
    if predicted_labels[i]==0:
      print('Fake')
    else:
      print('Not fake')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Article: The stock market is crashing today!
Prediction: 0 (Probability: 0.0057)
Fake
Article: NASA discovered water on Mars.
Prediction: 0 (Probability: 0.0025)
Fake
